### Zigbang 원룸 매물 데이터 수집

In [1]:
import requests
import pandas as pd

- 절차
    - 동이름으로 위도 경도 구하기
    - 위도 경도로 geohash 알아내기
    - geohash로 매물 아이디 가져오기
    - 매물 아이디로 매물 정보 가져오기

In [2]:
# 1. 동이름으로 위도 경도 구하기

In [3]:
addr = "망원동"
url = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸"
response = requests.get(url)
data = response.json()["items"][0]
lat, lng = data["lat"], data["lng"]
lat, lng

(37.556785583496094, 126.9013442993164)

In [4]:
# 2. 위도 경도로 geohash 알아내기

In [6]:
# install geohash2
# !pip install geohash2

In [7]:
import geohash2

In [8]:
# precision이 커질수록 영역이 작아짐
geohash = geohash2.encode(lat, lng, precision=5)
geohash

'wydjx'

In [9]:
# 3. geohash로 매물 아이디 가져오기

In [10]:
url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세\
&service_type_eq=원룸"
response = requests.get(url)
datas = response.json()["items"]
# len(datas), datas[0]
ids = [data["item_id"] for data in datas]
len(ids), ids[:5]

(825, [32090456, 32503839, 32508972, 32417712, 32504920])

In [11]:
# 4. 매물 아이디로 매물 정보 가져오기

In [12]:
# 1000개 넘어가면 나눠서 수집해야 함
url = "https://apis.zigbang.com/v2/items/list"
params = {
    "domain": "zigbang",
    "withCoalition": "true",
    "item_ids": ids
}
response = requests.post(url, params)
response

<Response [200]>

In [13]:
datas = response.json()["items"]
df = pd.DataFrame(datas)
df.tail(2)

,section_type,item_id,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,공급면적,전용면적,...,is_zzim,status,service_type,tags,address1,address2,address3,manage_cost,reg_date,is_new
823,None,32500166,https://ic.zigbang.com/ic/items/32500166/1.jpg,전세,전세,43000,0,31.74,"{'m2': 31.74, 'p': '9.6'}","{'m2': 27.13, 'p': '8.2'}",...,False,True,빌라,[],서울시 마포구 중동,None,None,5,2022-07-07T11:33:58+09:00,False
824,None,32171352,https://ic.zigbang.com/ic/items/32171352/1.jpg,월세,월세,5000,60,41.36,"{'m2': 41.36, 'p': '12.5'}","{'m2': 36.36, 'p': '11'}",...,False,True,빌라,[추천],서울시 마포구 중동,None,None,8,2022-06-25T14:54:20+09:00,False


In [14]:
# 필요한 컬럼만 필터링
columns = ["item_id", "sales_type", "deposit", "rent", "size_m2", "floor", "building_floor",
           "address1", "manage_cost"]
filtered_column_df = df[columns]
filtered_column_df.tail(2)

,item_id,sales_type,deposit,rent,size_m2,floor,building_floor,address1,manage_cost
823,32500166,전세,43000,0,31.74,3,6,서울시 마포구 중동,5
824,32171352,월세,5000,60,41.36,3,7,서울시 마포구 중동,8


In [15]:
# 주소에 망원동이 있는 데이터만 필터링
result_df = filtered_column_df[filtered_column_df["address1"].str.contains("망원동")]
result_df = result_df.reset_index(drop=True)
result_df.tail(2)

,item_id,sales_type,deposit,rent,size_m2,floor,building_floor,address1,manage_cost
112,32429265,월세,2000,65,20.54,4,5,서울시 마포구 망원동,2
113,32328328,월세,1000,45,19.83,5,5,서울시 마포구 망원동,5
